# Bandit Algorithms

[Look at this excellent resource for learning more about bandits algorithms](https://banditalgs.com/2018/02/09/bandit-tutorial-slides-and-update-on-book/)

author: Javier Marín, <p>
email: javier@jmarin.info,<p>
title: Reinforcement Learning is not just for playing games,<p>
subtitle: How can multi-armed bandits can help a startup



## Implementation

In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.figure_factory as ff
pio.templates.default = "plotly_white"

### Define the environment

In [2]:
m_size = ([0.11, 0.03, 0.08, 0.05, 0.16, 0.13, 0.06, 0.16, 0.03, 0.19])
np.sum(m_size)

1.0

In [3]:
# Group data together
hist_data = [m_size]
group_labels = ['Market size',
               ]
fig = ff.create_distplot(hist_data, 
                         group_labels, 
                         bin_size=0.01)
fig.update_xaxes(title='Normalized value')
fig.update_yaxes(title='Density')
fig.update_layout(plot_bgcolor='aliceblue')
fig.show()

In [4]:
class BanditEnv:
    def __init__(self):
        self.size = 10                                       
        self.means = m_size
    def step(self, action):
        assert action == int(action)
        assert 0 <= action < self.size        
        reward = np.random.normal(loc=self.means[action], scale=self.means[action])    
        return reward

### Epsilon Greedy

In [5]:
def epsilon_greedy(env, nb_total, eps, eps_factor=1.0):            
    rewards = []   
    Q = np.zeros(env.size)   # average reward for each arm, shape: [n_arms]
    N = np.zeros(env.size)   # num times each arm pulled,   shape: [n_arms]
    
    for _ in range(nb_total):
        if np.random.rand() > eps:
            A = np.random.choice(np.flatnonzero(Q == Q.max())) # pick best known arm
        else:
            A = np.random.randint(env.size)  # explore randomly
        R = env.step(A)
        N[A] += 1
        Q[A] += (1/N[A]) * (R - Q[A])        # incremental mean
        
        eps *= eps_factor                    # epsilon decay
        rewards.append(R)
        
    return Q, np.array(rewards)

In [6]:
env = BanditEnv()
nb_total=1000

Create and solve environment

In [7]:
epsilon = 0
Q1, rewards1 = epsilon_greedy(env, nb_total=nb_total, eps=epsilon)

In [8]:
epsilon2 = 0.1
Q1_1, rewards1_1 = epsilon_greedy(env, nb_total=nb_total, eps=epsilon2)

In [9]:
epsilon3 = 0.5
Q1_2, rewards1_2 = epsilon_greedy(env, nb_total=nb_total, eps=epsilon3)

In [10]:
fig = go.Figure(go.Scatter(x=np.arange(0,10), 
                       y=m_size, 
                       mode='markers',
                       marker_color='navy',
                       marker_symbol='hash-open-dot',
                       name='From benchmark analisys'))
fig.add_trace(
    go.Scatter(
        x=np.arange(0,10),
        y=Q1_1,
        mode='markers',
        marker_color='crimson',
        marker_symbol='triangle-down-open-dot',
        name = 'Predicted by agent'
    ))
fig.update_layout(
    title=f'Sales by product prediction; agent with Epsilon Greedy  ε={epsilon2} and {nb_total} time-stpes',
    xaxis=dict(
        title='Accesories',
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis=dict(
        title='% of sales',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    bargap=0.90, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.update_xaxes(nticks=10, tickvals=[0,1,2,3,4,5,6,7,8,9])
fig.update_layout(plot_bgcolor='aliceblue')
fig.show()

In [12]:
fig = go.Figure(go.Scatter(x=np.arange(0,nb_total), 
                           y=np.cumsum(rewards1),
                           mode='lines',
                           name=f'With Epsilon Greedy ε={epsilon}'))
fig.add_trace(
    go.Scatter(
        x=np.arange(0,nb_total),
        y=np.cumsum(rewards1_1),
        mode='lines',
        name = f'With Epsilon Greedy ε={epsilon2}'
    ))

fig.add_trace(
    go.Scatter(
        x=np.arange(0,nb_total),
        y=np.cumsum(rewards1_2),
        mode='lines',
        name = f'With Epsilon Greedy ε={epsilon3}'
    ))
fig.update_layout(title=f'Cumulative Reward comparison after {nb_total} time-steps')
fig.update_xaxes(title='Time steps')
fig.update_yaxes(title='Cumulative Reward')
fig.update_layout(plot_bgcolor='aliceblue')
fig.show()

In [13]:
cum_regrets1 = np.cumsum(max(env.means) - rewards1)
cum_regrets2 = np.cumsum(max(env.means) - rewards1_1)
cum_regrets3 = np.cumsum(max(env.means) - rewards1_2)

fig = go.Figure(go.Scatter(x=np.arange(0,nb_total), 
                           y=cum_regrets1,
                           mode='lines',
                           name=f'With Epsilon Greedy ε={epsilon}'))
fig.add_trace(
    go.Scatter(
        x=np.arange(0,nb_total),
        y=cum_regrets2,
        mode='lines',
        name = f'With Epsilon Greedy ε={epsilon2}'
    ))

fig.add_trace(
    go.Scatter(
        x=np.arange(0,nb_total),
        y=cum_regrets3,
        mode='lines',
        name = f'With Epsilon Greedy ε={epsilon3}'
    ))
fig.update_layout(title=f'Cumulative Regrets comparison')
fig.update_xaxes(title='Time steps')
fig.update_yaxes(title='Cumulative Regret')
fig.update_layout(plot_bgcolor='aliceblue')
fig.show()

### Upper Confidence Bound

In [14]:
def upper_confidence_bound(env, nb_total, c_param):
    
    rewards = []
    
    V = np.zeros(env.size)   # sum of reward for each arm, shape: [n_arms]
    N = np.zeros(env.size)   # num times each arm pulled,   shape: [n_arms]
    
    for k in range(nb_total):
        
        if k < env.size:     # visit each arm at least once
            A = k
        else:                # the famous UCB formula in vectorised form
            scores = V / N + c_param * np.sqrt( np.log(N.sum()) / N )
            A = np.random.choice(np.flatnonzero(scores == scores.max()))
        
        R = env.step(A)
        N[A] += 1
        V[A] += R

        rewards.append(R)
        
    Q = V / N                # calculate arm values before returning
    return Q, np.array(rewards)

Note that top nodes are very tightly approximated, while poorest nodes are only vaguely approximated. This is because UCB actively focuses on best nodes early (but keeps exploring a bit).

In [15]:
c_param = np.sqrt(2)
Q2, rewards2 = upper_confidence_bound(env, nb_total=nb_total, c_param=c_param)

In [16]:
fig = go.Figure(go.Scatter(x=np.arange(0,10), 
                       y=m_size, 
                       mode='markers',
                       marker_color='navy',
                       marker_symbol='hash-open-dot',
                       name='From benchmark analisys'))
fig.add_trace(
    go.Scatter(
        x=np.arange(0,10),
        y=Q2,
        mode='markers',
        marker_color='crimson',
        marker_symbol='triangle-down-open-dot',
        name = 'Predicted by agent'
    ))
fig.update_layout(
    title=f'Sales by product prediction agent with Upper Confidence Bound (c={c_param:.2f})',
    xaxis=dict(
        title='Accesories',
        titlefont_size=16,
        tickfont_size=14,
    ),
    yaxis=dict(
        title='% of sales',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    bargap=0.90, # gap between bars of adjacent location coordinates.
    bargroupgap=0.1 # gap between bars of the same location coordinate.
)
fig.update_xaxes(nticks=10, tickvals=[0,1,2,3,4,5,6,7,8,9])
fig.update_layout(plot_bgcolor='aliceblue')
fig.show()

Plot cumulative regret

In [17]:
cum_regrets4 = np.cumsum(max(env.means)-rewards2)
fig = px.line(x=np.arange(0, nb_total), y= cum_regrets4)
fig.update_layout(title=f'Cumulative Regret with c={c_param:.2f}')
fig.update_xaxes(title='Time steps')
fig.update_yaxes(title='Regret')
fig.update_layout(plot_bgcolor='aliceblue')
fig.show()

In [18]:
fig = go.Figure(go.Scatter(x=np.arange(0,nb_total), 
                           y= cum_regrets2,
                           mode='lines',
                           line_color='navy',
                           name=f'With Epsilon Greedy (ε={epsilon2})'))
fig.add_trace(
    go.Scatter(
        x=np.arange(0,nb_total),
        y=cum_regrets4,
        mode='lines',
        line_color='crimson',
        name = f'With Upper Confidence Bound (c={c_param:.2f})'
    ))
fig.update_layout(title=f'Cumulative Regret comparison')
fig.update_xaxes(title='Time steps')
fig.update_yaxes(title='Regret')
fig.update_layout(plot_bgcolor='aliceblue')
fig.show()